<a href="https://colab.research.google.com/github/ljkrajewski/ollama-local-rag/blob/main/RAG_colab_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title User Input
#@markdown Enter LLM and embedding models
LLM_MODEL="tinyllama" #@param string
EMBEDDING_MODEL="mxbai-embed-large" #@param string
#@markdown Enter github.com repository of Markdown docs
REPO_URL="https://github.com/ljkrajewski/echoes_of_orpheus.git" #@param string

In [2]:
#@title Install ollama and download the tinyllama LLM
!curl -fsSL https://ollama.com/install.sh | sh

import multiprocessing
import os
import time

def run_ollama():
    print(f"Running ollama on PID {os.getpid()}")
    os.system('ollama serve')

ollama_process = multiprocessing.Process(target=run_ollama)
ollama_process.start()
time.sleep(10)

!ollama pull {LLM_MODEL}
!ollama pull {EMBEDDING_MODEL}

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Running ollama on PID 875




In [4]:
#@title Install ollama-local-rag
!pip install numpy==2.2.6, packaging>=24.2.0, langchain>=0.3.0, langchain-community>=0.3.0
!pip install python-dotenv
!git clone https://github.com/ljkrajewski/ollama-local-rag.git
%cd ollama-local-rag
ENVFILE=f"""
LLM_MODEL={LLM_MODEL}
EMBEDDING_MODEL={EMBEDDING_MODEL}
"""
with open('.env', 'w') as f:
    f.write(ENVFILE)
!pip install -r requirements.txt

import nltk
nltk.download('all')

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.16 requires numpy<2,>=1, but you have numpy 2.2.6 which is incompatible.
langchain-community 0.0.33 requires numpy<2,>=1, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.
db-dtypes 1.4.3 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.
Cloning into 'ollama-local-rag'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 115 (delta 61), reused 25 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (115/115), 1.27 MiB | 21.01 MiB/s, done.
Resolving deltas: 100% (61/61), done.
/content/o

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_r

True

In [ ]:
#@title Load files to populate the database
from google.colab import files

%cd /content/ollama-local-rag/docs
!rm *
!git clone $REPO_URL

"""
# Upload files to index
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
"""

%cd /content/ollama-local-rag
!python create_database.py

In [ ]:
#@title Start gradio
!pip install -q gradio

import gradio as gr
import re
import subprocess
import sys

def generate(prompt):
  """
  Runs a separate Python script and returns its output.

  Args:
      script_path (str): The path to the Python script to execute.
      *args: Any arguments to pass to the script.

  Returns:
      tuple: A tuple containing the script's standard output (stdout) and standard error (stderr) as strings,
              or None if an error occurred.
  """
  script_path = "/content/ollama-local-rag/query_data.py"
  args = [prompt]
  try:
    # Construct the command to run the script
    command = [sys.executable, script_path] + list(args)

    # Run the script and capture its output
    process = subprocess.run(command, capture_output=True, text=True, check=True)

    # Return the stdout and stderr
    return process.stdout, process.stderr

  except subprocess.CalledProcessError as e:
    # Handle errors from the subprocess
    print(f"Error running script: {e}")
    return e.stdout, e.stderr #Return the error output if the subprocess failed.

  except FileNotFoundError:
    print(f"Script not found: {script_path}")
    return None, None

  except Exception as e:
    print(f"An unexpected error occurred: {e}")
    return None, None

with gr.Blocks(analytics_enabled=False) as demo:
  with gr.Column():
    with gr.Row():
      prompt_box = gr.Textbox(lines=10, interactive=True, value="", label="Prompt")
      with gr.Column():
        generate_button = gr.Button("Generate")
        log_box = gr.Textbox(lines=3, interactive=False, label="System Messages")
    result_box = gr.Markdown(show_copy_button=True, label="Result", show_label=True, container=True, line_breaks=True, height="1000")
  generate_button.click(fn=generate, inputs=[prompt_box], outputs=[result_box, log_box])

demo.queue().launch(inline=False, share=True, debug=True)